In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Fetching Data

In [ ]:
df_tl = pd.read_csv('./data/ThorLabs_Lenses.csv')
df_tl

In [ ]:
df_eo = pd.read_csv('./data/Edmund-Optics_Lenses.csv')
df_eo

In [ ]:
# Combine both dataframes and save to CSV
df = pd.concat([df_tl, df_eo], ignore_index=True)
df.to_csv('./data/Combined_Lenses.csv', index=False)
df

In [ ]:
# Unique radii
rad_ll = np.array(df['Diameter (mm)'].unique()) / 2
print("Radii (mm):", rad_ll)

# Finding Candidates for Lens 1

In [ ]:
z_0 = 11.01 # mm

# Optimal focal lengths for 33 degree divergence
optimal_f = np.array([ rad/np.tan(np.deg2rad(33)) for rad in rad_ll ])
print("Optimal Focal Lengths (mm):", optimal_f)

# Finding the closest available focal lengths in the dataset
available_f = df['Focal Length (mm)'].values
closest_f = np.array([ min(available_f, key=lambda x: abs(x - opt_f)) for opt_f in optimal_f ])

# Remove items that are smaller than z_0
closest_f = closest_f[closest_f >= z_0]
print("Closest Focal Lengths (mm):", closest_f)

# Finding all lens Item # for these focal lengths
lens_items = [df[df['Focal Length (mm)'] == f]['Item #'].tolist() for f in closest_f]
# Convert list of lists to a flat list and remove duplicates
lens_items = list(set(item for sublist in lens_items for item in sublist))
print("Lens Item Numbers:", lens_items)

# z values for these focal lengths
z_values = closest_f - z_0
print("z values (mm):", z_values)

In [ ]:
# Fetch data for lens_items from the dataframe as a dataframe
lens_data = df[df['Item #'].isin(lens_items)]
lens_data = lens_data.sort_values(by='Diameter (mm)')
lens_data.reset_index(drop=True, inplace=True)
lens_data

In [ ]:
# Save lens data to CSV
lens_data.to_csv('./data/l1_candidates.csv', index=False)

# Finding Candidates for Lens 2

In [ ]:
acceptance_angle = np.deg2rad(12.4)  # 33 degrees in radians

# Find z for each lens
# z = radius / tan(acceptance_angle)
z_values = rad_ll / np.tan(acceptance_angle)
print("z values (mm):", z_values)

optimal_f = z_values

# Finding the closest available focal lengths in the dataset
available_f = df['Focal Length (mm)'].values
closest_f = np.array([ min(available_f, key=lambda x: abs(x - opt_f)) for opt_f in optimal_f ])

# Finding all lens Item # for these focal lengths
lens_items = [df[df['Focal Length (mm)'] == f]['Item #'].tolist() for f in closest_f]
# Convert list of lists to a flat list and remove duplicates
lens_items = list(set(item for sublist in lens_items for item in sublist))
print("Lens Item Numbers:", lens_items)

In [ ]:
# Fetch data for lens_items from the dataframe as a dataframe
lens_data = df[df['Item #'].isin(lens_items)]
lens_data = lens_data.sort_values(by='Diameter (mm)')
lens_data.reset_index(drop=True, inplace=True)
lens_data

In [ ]:
# Save lens data to CSV
lens_data.to_csv('./data/l2_candidates.csv', index=False)

### Used to prepare dataframe for use. No longer needed

In [ ]:
# df_tl = pd.read_csv('./data/ThorLabs_Lenses.csv').iloc[:, :-1]

# def clean_value(val):
#     if not isinstance(val, str):
#         return val
#     if 'mm' in val:
#         return float(val.replace(' mm', ''))
#     if '"' in val:
#         val = val.replace('"', '')
#         return float(eval(val)) * 25.4 if '/' in val else float(val) * 25.4
#     return val

# unit_columns = ['Diameter', 'Focal Length', 'Radius of Curvature', 
#                 'Center Thickness', 'Edge Thickness', 'Back Focal Length']

# for col in unit_columns:
#     df_tl[col] = df_tl[col].apply(clean_value)
#     df_tl = df_tl.rename(columns={col: f'{col} (mm)'})

# df_tl = df_tl.rename(columns={'Item #a': 'Item #', 'Diopterb': 'Diopter'})
# # df_tl

# # Delete the Diopter column (not needed)
# df_tl = df_tl.drop(columns=['Diopter'])
# df_tl.to_csv('./data/ThorLabs_Lenses.csv', index=False)

# # Override existing file
# df_tl.to_csv('./data/ThorLabs_Lenses.csv', index=False)

In [ ]:
# df_eo = pd.read_excel('./data/Edmund-Optics_Lenses.xlsx').iloc[:, [1,3,4,12,13,14,15]]

# # Renamed Stock Number as Item #
# # Used Clear Aperture (CA) as Diameter
# # Truncated the error bars
# # Used Effective Focal Length (EFL) as Focal Length

# def clean_value(val):
#     if not isinstance(val, str):
#         return val
#     if '±' in val:
#         val = val.split('±')[0].strip()
#     if '@' in val:
#         val = val.split('@')[0].strip()
#     return float(val)

# unit_columns = ['CA (mm)', 'EFL (mm)', 'Radius R1 (mm)',
#                 'CT (mm)', 'ET (mm)', 'BFL (mm)']

# for col in unit_columns:
#     df_eo[col] = df_eo[col].apply(clean_value)

# # Rename columns to match ThorLabs format
# df_eo = df_eo.rename(columns={
#     'Stock Number': 'Item #',
#     'EFL (mm)': 'Focal Length (mm)',
#     'BFL (mm)': 'Back Focal Length (mm)',
#     'CT (mm)': 'Center Thickness (mm)',
#     'ET (mm)': 'Edge Thickness (mm)',
#     'Radius R1 (mm)': 'Radius of Curvature (mm)',
#     'CA (mm)': 'Diameter (mm)',
# })

# df_eo = df_eo[['Item #', 'Diameter (mm)', 'Focal Length (mm)', 'Radius of Curvature (mm)', 'Center Thickness (mm)', 'Edge Thickness (mm)', 'Back Focal Length (mm)']]

# # Write to CSV
# df_eo.to_csv('./data/Edmund-Optics_Lenses.csv', index=False)